# Day 4: Security Through Obscurity

## Part 1: 
> Each room consists of an encrypted name (lowercase letters separated by dashes) followed by a dash, a sector ID, and a checksum in square brackets.

> A room is real (not a decoy) if the checksum is the five most common letters in the encrypted name, in order, with ties broken by alphabetization.

In [2]:
from utils import *
from collections import Counter

rooms_input = read_input_as_list('day4', '')
head(rooms_input)

['hqcfqwydw-fbqijys-whqii-huiuqhsx-660[qhiwf]', 'oxjmxdfkd-pzxsbkdbo-erkq-ixyloxqlov-913[xodkb]', 'bpvctixr-eaphixr-vgphh-gthtpgrw-947[smrkl]', 'iwcjapey-lhwopey-cnwoo-wymqeoepekj-992[eowpy]']


In [25]:
def get_most_common_chars(chars, n):
    dist = Counter(sorted(cat(chars)))
    return [t[0] for t in dist.most_common(n)]

def room_is_valid(chars, checksum):
    common_letters = cat(get_most_common_chars(chars, 5))
    return common_letters == checksum

assert room_is_valid('aaaaabbbzyx', 'abxyz')
assert not room_is_valid('totallyrealroom', 'decoy')

def part1(rooms):
    sector_id_sum = 0
    for room in rooms:
        *chars, code = room.split('-')
        sector_id, checksum = code.split('[')
        checksum = checksum[:-1] # remove the trailing ]
        if room_is_valid(chars, checksum):
            sector_id_sum += int(sector_id)
    return sector_id_sum

assert part1(['aaaaa-bbb-z-y-x-123[abxyz]']) == 123 

part1(rooms_input)

245102

## Part 2

> The room names are encrypted by a state-of-the-art shift cipher, which is nearly unbreakable without the right software. However, the information kiosk designers at Easter Bunny HQ were not expecting to deal with a master cryptographer like yourself.

> To decrypt a room name, rotate each letter forward through the alphabet a number of times equal to the room's sector ID. A becomes B, B becomes C, Z becomes A, and so on. Dashes become spaces.

The success condition is described as being:
>What is the sector ID of the room where North Pole objects are stored?

Which is a little confusing, but maybe it'll be clearer after implementing the decryption function. The encryption function is essentially `rot_n`, which should be straightforward. This is good opportunity to use the `transform` and `maketrans` methods in `str` as well.

In [43]:
import string

def rot_n(rot):
    rot = rot % 26
    letters = string.ascii_lowercase # only care about lowercase letters here
    tt =  str.maketrans(letters, letters[rot:] + letters[:rot]) # construct the lookup table 
    return lambda s: s.translate(tt) 
    
assert rot_n(343)('qzmt') == 'very'

def decrypt_room(strings, rot):
    return list(map(rot_n(rot), strings))

assert decrypt_room(['qzmt', 'zixmtkozy', 'ivhz'], 343) == ['very', 'encrypted', 'name']

def part2(rooms):
    for room in rooms:
        *chars, code = room.split('-')
        sector_id, checksum = code.split('[')
        checksum = checksum[:-1] # remove the trailing ]
        if room_is_valid(chars, checksum):
            if 'northpole' in decrypt_room(chars, int(sector_id)):
                print(sector_id)

part2(rooms_input)

324


I still don't really get how I was supposed to know the exact string was `northpole`. I just printed all of them then searched for the string `north`. Still, a win is a win.